In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

from datetime import datetime

### 1 - Import test data

In [3]:
dataF = yf.download("EURUSD=X", start="2024-10-05", end="2024-12-02", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,,
2024-10-07 00:00:00,1.097333,1.097333,1.097333,1.097213,1.097333,0
2024-10-07 00:15:00,1.097213,1.097213,1.097333,1.097213,1.097213,0
2024-10-07 00:30:00,1.097333,1.097333,1.097333,1.097213,1.097213,0
2024-10-07 00:45:00,1.097454,1.097454,1.097454,1.097093,1.097454,0
2024-10-07 01:00:00,1.097454,1.097454,1.097454,1.097333,1.097333,0
...,...,...,...,...,...,...
2024-11-29 21:15:00,1.058201,1.058201,1.058313,1.058089,1.058313,0
2024-11-29 21:30:00,1.058201,1.058201,1.058425,1.058201,1.058201,0


In [4]:
dataF.columns = dataF.columns.droplevel(1)

In [5]:
dataF.reset_index(inplace=True)

In [6]:
display(dataF)

Price,Datetime,Adj Close,Close,High,Low,Open,Volume
0,2024-10-07 00:00:00,1.097333,1.097333,1.097333,1.097213,1.097333,0
1,2024-10-07 00:15:00,1.097213,1.097213,1.097333,1.097213,1.097213,0
2,2024-10-07 00:30:00,1.097333,1.097333,1.097333,1.097213,1.097213,0
3,2024-10-07 00:45:00,1.097454,1.097454,1.097454,1.097093,1.097454,0
4,2024-10-07 01:00:00,1.097454,1.097454,1.097454,1.097333,1.097333,0
...,...,...,...,...,...,...,...
3782,2024-11-29 21:15:00,1.058201,1.058201,1.058313,1.058089,1.058313,0
3783,2024-11-29 21:30:00,1.058201,1.058201,1.058425,1.058201,1.058201,0
3784,2024-11-29 21:45:00,1.058089,1.058089,1.058537,1.058089,1.058537,0
3785,2024-11-29 22:00:00,1.058089,1.058089,1.058089,1.058089,1.058089,0


In [7]:
# Calculate the 50-day EMA
dataF['EMA_50'] = dataF['Adj Close'].ewm(span=50, adjust=False).mean()

In [8]:
fig = go.Figure(data=[go.Candlestick(x= dataF.index, # dataF["Datetime"]
                open=dataF['Open'],
                high=dataF['High'],
                low=dataF['Low'],
                close=dataF['Close'])])

fig.add_trace(go.Scatter(
    x= dataF.index, # dataF["Datetime"]
    y=dataF['EMA_50'],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig.update_layout(height=800)

fig.show()

In [9]:
print(dataF["EMA_50"].iloc[-1])

1.0571601060488889


### 2 - Define your signal function

#### We define the buying strategy

In [ ]:
# Note that in between all these functions time can flow (for example between buy and get_swing_high_point_before_pullback) or no time can flow,
# i.e. the functions are executed in the same time frame (for example between pullback and get_swing_high_point_before_pullback there is no time 
# in between, the functions are executed one after the other).

In [10]:
def price_below_EMA(df):
    # Is True if price is below 50 day EMA 
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    if open >= EMA_50 and close <= EMA_50:
        return True
    
    return False

def strategy_start(df):
    # Strategy starts once the price is above EMA and a candle closes above it
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    if close >= EMA_50:
        return True
    
    return False

def pullback(df):
    # We consider a Pullback when we have at least 2 opposite candles coming down
    bool1 = (df.Open.iloc[-1] - df.Close.iloc[-1] >= 0)
    bool2 = (df.Open.iloc[-2] - df.Close.iloc[-2] >= 0)

    if bool1 and bool2:
        return True
    
    return False

def get_swing_high_point_before_pullback(df):
    # We apply this function only if the other one is True

    # Gets high point before pullback
    high_point = df.High.iloc[-3]

    len_high_point_candle = df.Close.iloc[-3] - df.Open.iloc[-3]

    return high_point, len_high_point_candle

def invalid_trade_1(df):
    # Check this condition before buying

    # If the prices closes below the EMA after the pullback then the trade is invalid
    if df.Close.iloc[-1] < df["EMA_50"].iloc[-1]:
        # Trade is invalid
        return True
    # trade is valid
    return False

def invalid_trade_2(df, len_high_point_candle):
    # This functions needs to be called right after get_swing_high_point_before_pullback
    # Calculate tthe mean height of the last 40 candles

    # If the breakcoutcandle is 3 or 4 times bigger than the candles before, then the trade is invalid
    df['AbsDiff'] = (df.Close - df.Open).abs()
    mean_abs_diff = df['AbsDiff'].iloc[:50].mean()

    if len_high_point_candle >= 3*mean_abs_diff:
        return True
    return False


def buy(df, high_point):
    # The body of the candle needs to close above the swing high point
    close = df.Close.iloc[-1]

    if close >= high_point:
        return True
    
    return False

In [ ]:
# The bots need to be able to do 2 things, see future trading strateegies, and if it is started in one it should be able to work from there
# For this second option I need to go row by row and append to some available data and then run the function below on these partial dataframes
# I will also do this for the testing now but it has to be done once I am connected to the broker also

# I need to implement take proft target

In [37]:
# Do on running data
def trading_strategy(df):
    global cond_1 
    global cond_2 
    global cond_3 

    global step_1
    global step_2
    global step_3

    if step_1 and price_below_EMA(df):
        print("test1")
        cond_1 = True
        step_1 = False
    if step_2 and cond_1 and strategy_start(df):
        print("test2")
        cond_2 = True
        step_2 = False
    if step_3 and cond_1 and cond_2 and pullback(df):
        print("test3")
        cond_3 = True
        step_3 = False

    if cond_3:
        high_point, len_high_point_candle = get_swing_high_point_before_pullback(df)

    if cond_3 and buy(df, high_point):
        print("BUY")
        # Restart again
        step_1, step_2, step_3 = True, True, True
        cond_1, cond_2, cond_3 = False, False, False
        return "buy"

    elif cond_3 and invalid_trade_1(df):
        print("Invalid trade1")
        step_1, step_2, step_3 = True, True, True
        cond_1, cond_2, cond_3 = False, False, False
        return "invalid1"
    
    elif cond_3 and invalid_trade_2(df, len_high_point_candle):
        print("Invalid trade2")
        step_1, step_2, step_3 = True, True, True
        cond_1, cond_2, cond_3 = False, False, False
        return "invalid2"
    
    else:
        return "nada"
    

In [12]:
test = dataF[dataF["Datetime"] >= "2024-11-19"]

In [16]:
fig2 = go.Figure(data=[go.Candlestick(x= test.index, # dataF["Datetime"]
                open=test['Open'],
                high=test['High'],
                low=test['Low'],
                close=test['Close'])])

fig2.add_trace(go.Scatter(
    x= test.index, # dataF["Datetime"]
    y=test['EMA_50'],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig2.update_layout(height=800)

fig2.show()

In [ ]:
print(test.dtypes)

In [13]:
available_df = pd.DataFrame(columns=test.columns)

In [38]:
cond_1 = False
cond_2 = False
cond_3 = False

step_1 = True
step_2 = True
step_3 = True

for index, row in test.iterrows():
    row_df = pd.DataFrame([row])
    available_df = pd.concat([available_df, row_df])
    print(len(available_df))
    if len(available_df) >= 3:
        res = trading_strategy(available_df)
        if res == "buy":
            available_df = pd.DataFrame(columns=test.columns)
        elif res == "invalid1":
            available_df = pd.DataFrame(columns=test.columns)
        elif res == "invalid2":
            available_df = pd.DataFrame(columns=test.columns)
        else:
            trading_strategy(available_df)

9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
test1
29
30
31
32
33
34
35
test2
36
37
38
39
40
41
42
43
44
test3
Invalid trade1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
test1
25
26
test2
27
28
29
30
31
test3
32
33
BUY
1
2
3
4
test1
5
6
test2
7
8
9
test3
10
11
BUY
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
test1
32
33
34
35
36
37
38
39
40
41
42
43
test2
44
45
46
47
test3
Invalid trade1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
test1
68
test2
69
70
71
72
73
74
75
76
test3
BUY
1
2
3
4
5
6
7
8
9
10
11
12
13
test1
14
test2
15
16
test3
17
Invalid trade1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
test1
30
31
32
test2
33
34
test3
BUY
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
3

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

31
32
33
34
35
36
37
38
39
40
41
42
43
44
test1
45
46
47
test2
48
49
test3
Invalid trade1
1
2
3
4
5
6
7
8
9
10
11
12
13
test1
14
15
test2
16
17
18
19
20
21
22
test3
Invalid trade2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
test1
65
66
test2
67
68
test3
69
70
71
Invalid trade1
1
2
3
4
test1
5
6
7
test2
8
9
test3
Invalid trade1
1
2
3
4
test1
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
test2
25
26
27
test3
28
Invalid trade1
1
2
3
4
test1
5
6
7
8
9
10
11
12
test2
13
14
15
test3
Invalid trade1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
test1
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
test2
95
96
97
98
99
100
101
102
103
104
105
106
107
108
test3
109
BUY
1
2
3

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/6b/r3xhr4d94wn52lq9yxzbnnzm0000gn/T/ipykernel_960/1583424541.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

In [ ]:
display(available_df)

### Rest of code, not done yet

In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()